In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [2]:
def save_img(imgs,names):
  img_new = Image.new('L',(280,280))
  index = 0
  for i in range(0,280,80):
    for j in range(0,280,80):
      img = imgs[index]
      img = Image.fromarray(img,mode='L')
      img_new.paste(img,(i,j))
      index+=1
  img_new.save(names)

In [3]:
def feature_scale(x):
  x = tf.cast(x,dtype=tf.float32)/255.
#  y = tf.cast(y,dtype=tf.int32)
  return x

In [4]:
#Dim reduct nums
dim_reduce = 10
batch_num = 128
lr = 1e-3

In [5]:
(x,y),(x_test,y_test) = datasets.fashion_mnist.load_data()
data = tf.data.Dataset.from_tensor_slices(x)
data = data.map(feature_scale).shuffle(10000).batch(128)

data_test = tf.data.Dataset.from_tensor_slices(x_test)
data_test = data_test.map(feature_scale).batch(128)
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

4423680/4422102 [==============================] - 0s 0us/step
(28, 28) (28, 28)


In [6]:
class AE(keras.Model):
  def __init__(self):
    super(AE,self).__init__()
    #encoder
    self.model_encoder = Sequential([layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(dim_reduce,activation=tf.nn.relu),
                            ])
    
    #decoder
    self.model_decoder = Sequential([layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(784,activation=tf.nn.relu), 
                            ])

  def call(self, inputs, training=None):
    x = self.model_encoder(inputs)
    x = self.model_decoder(x)
    return x

In [7]:
model = AE()
model.build(input_shape=(None, 784))
model.summary()

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 10)                235146    
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               235920    
Total params: 471,066
Trainable params: 471,066
Non-trainable params: 0
_________________________________________________________________


In [9]:
!rm -rf img_result
!mkdir img_result

In [10]:
optimizer = optimizers.Adam(lr=lr)

for i in range(10):
  for step,x in enumerate(data):
    x = tf.reshape(x,[-1,784])
    with tf.GradientTape() as tape:
      logits = model(x)
      loss = tf.losses.binary_crossentropy(x,logits,from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss))
      
  x = next(iter(data_test))
  val_x = tf.reshape(x,[-1,784])
  logits = model(val_x)
  x_hat = tf.sigmoid(logits)
  x_hat = tf.reshape(x_hat,[-1,28,28])
  x_hat = x_hat.numpy()*255
  x_hat = x_hat.astype(np.uint8)
  save_img(x_hat,'img_result/AE_img_%d.png'%i)

0 0 loss: 0.6802642345428467
0 100 loss: 0.6781050562858582
0 200 loss: 0.679611325263977
0 300 loss: 0.6795214414596558
0 400 loss: 0.6809988021850586
1 0 loss: 0.6787962913513184
1 100 loss: 0.6802211403846741
1 200 loss: 0.6817357540130615
1 300 loss: 0.6767575740814209
1 400 loss: 0.6781867742538452
2 0 loss: 0.679638683795929
2 100 loss: 0.6779047250747681
2 200 loss: 0.673526406288147
2 300 loss: 0.6684237718582153
2 400 loss: 0.6605972051620483
3 0 loss: 0.6658421158790588
3 100 loss: 0.6640974879264832
3 200 loss: 0.66722571849823
3 300 loss: 0.6639797687530518
3 400 loss: 0.6613168716430664
4 0 loss: 0.6594548225402832
4 100 loss: 0.6561526656150818
4 200 loss: 0.660636305809021
4 300 loss: 0.6588587760925293
4 400 loss: 0.6586166620254517
5 0 loss: 0.6635708808898926
5 100 loss: 0.6528977155685425
5 200 loss: 0.6676043272018433
5 300 loss: 0.6539176106452942
5 400 loss: 0.6532435417175293
6 0 loss: 0.6497045159339905
6 100 loss: 0.6570489406585693
6 200 loss: 0.65387278795242